##### <center>**CMPINF 0010 - Group 36 Final Project**</center>
## <center>**What is the Least Depressing Neighborhood in Pittsburgh?**</center>

##### **The Project**
The aim of this project is to quantitatively determine the Pittsburgh neighborhood that is the least depressing to live in. To achieve this, we will be providing a clearly defined metric that we believe may indicate this quality in a neighborhood. The metric will be quantitative in nature and will be sourced from the **Western Pennsylvania Regional Data Center (WPRDC)**.
Our approach is to pick three submetrics from the **WPRDC**, each of which we believe to have significance in our definition of *least depressing*, given below. We will analyze the submetrics individually, as well as considered together, to arrive at a verdict for overall least-depressiveness.

##### **The Metric**
We quantitatively define the "least depressing neighborhood" as the neighborhood that:
<br>
* `contains or has easy access to the largest number of city parks;`
<br>
* `has the lowest lifetime number of police arrests;`
<br>
* and `has the highest rates of community center attendance.`

##### **Coming Up with the Metric**
Our process of arriving at our decided metric was sophisticated. We threw around a handful of spontaneous ideas--most of which were not put to paper and thus unable to be recalled--until we found one we both seemed to be okay with--the "least depressing" metric for determining the best neighborhood.
<br>
Afterwards, we combed through the WPRDC website for available datasets that were also well-structured and thus not a hassle to analyze.
<br>
Out of the approximately nine suitable datasets, namely
* no. of trees
* no. of signalized intersections
* no. of city parks
* community center attendance
* no. of smart trash containers
* no. of playground
* no. of bridges (this one might cut across neighborhoods)
* no. of retaining walls
* no. of water features (fountains, etc.),

we settled on the current three that became our three submetrics that we will be analyzing individually and in tandem to fulfill our main metric.

##### **Analysis: Police Arrests**

In [23]:
import pandas as pd
pd.set_option('display.max_rows', None)

arrests = pd.read_csv('./arrests/arrests.csv')
arrests["INCIDENTNEIGHBORHOOD"].value_counts()

INCIDENTNEIGHBORHOOD
Central Business District      4440
South Side Flats               3497
Carrick                        2352
East Allegheny                 2217
Homewood South                 2090
Homewood North                 1969
East Liberty                   1889
Marshall-Shadeland             1516
Bluff                          1502
Mount Washington               1434
Hazelwood                      1396
Brookline                      1390
Lincoln-Lemington-Belmar       1362
Knoxville                      1336
Sheraden                       1285
Allegheny Center               1244
Beechview                      1201
East Hills                     1194
Brighton Heights               1114
Perry South                    1057
Bloomfield                      984
Shadyside                       971
Allentown                       947
Middle Hill                     932
Larimer                         879
Central North Side              853
Crawford-Roberts                839
North S

##### **Analysis: City Parks**

In [26]:
parks = pd.read_csv('./city-parks/city-parks.csv')
parks["neighborhood"].value_counts()

neighborhood
East Liberty                 12
Central Business District    10
Beechview                     8
South Side Slopes             7
Point Breeze                  7
Hazelwood                     6
Squirrel Hill South           6
South Side Flats              6
Mount Washington              6
Sheraden                      6
Troy Hill                     5
South Oakland                 5
Central Oakland               4
Elliott                       4
Marshall-Shadeland            4
Brighton Heights              4
Crawford-Roberts              3
Greenfield                    3
Larimer                       3
North Shore                   3
Central Northside             3
Upper Lawrenceville           3
Westwood                      3
Lower Lawrenceville           3
Lincoln-Lemington-Belmar      3
Garfield                      3
Brookline                     3
Carrick                       3
Spring Hill-City View         3
Point Breeze North            3
Swisshelm Park             

##### **Analysis: Community Center Attendance**

##### **Verdict: The Overall Least Depressing Neighborhood**

##### **Conclusion**